In [42]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
import sys

In [46]:
spark = SparkSession.builder.master("local[*]").appName("NFL").getOrCreate()

In [60]:
# injury_rec = sc.textFile('../in/InjuryRecord.csv')
# play_rec = sc.textFile('../in/PlayList.csv')
# sqlContext = SQLContext(sc)

injury_rec_df = spark.read.option("header",True).csv('../in/InjuryRecord.csv')
play_rec_df = spark.read.option("header",True).csv('../in/PlayList.csv')

In [61]:
injury_rec_df.take(10)

[Row(PlayerKey='39873', GameID='39873-4', PlayKey='39873-4-32', BodyPart='Knee', Surface='Synthetic', DM_M1='1', DM_M7='1', DM_M28='1', DM_M42='1'),
 Row(PlayerKey='46074', GameID='46074-7', PlayKey='46074-7-26', BodyPart='Knee', Surface='Natural', DM_M1='1', DM_M7='1', DM_M28='0', DM_M42='0'),
 Row(PlayerKey='36557', GameID='36557-1', PlayKey='36557-1-70', BodyPart='Ankle', Surface='Synthetic', DM_M1='1', DM_M7='1', DM_M28='1', DM_M42='1'),
 Row(PlayerKey='46646', GameID='46646-3', PlayKey='46646-3-30', BodyPart='Ankle', Surface='Natural', DM_M1='1', DM_M7='0', DM_M28='0', DM_M42='0'),
 Row(PlayerKey='43532', GameID='43532-5', PlayKey='43532-5-69', BodyPart='Ankle', Surface='Synthetic', DM_M1='1', DM_M7='1', DM_M28='1', DM_M42='1'),
 Row(PlayerKey='41145', GameID='41145-2', PlayKey='41145-2-60', BodyPart='Knee', Surface='Natural', DM_M1='1', DM_M7='0', DM_M28='0', DM_M42='0'),
 Row(PlayerKey='46014', GameID='46014-10', PlayKey='46014-10-22', BodyPart='Ankle', Surface='Synthetic', DM_M

In [62]:
play_rec_df.take(10)

[Row(PlayerKey='26624', GameID='26624-1', PlayKey='26624-1-1', RosterPosition='Quarterback', PlayerDay='1', PlayerGame='1', StadiumType='Outdoor', FieldType='Synthetic', Temperature='63', Weather='Clear and warm', PlayType='Pass', PlayerGamePlay='1', Position='QB', PositionGroup='QB'),
 Row(PlayerKey='26624', GameID='26624-1', PlayKey='26624-1-2', RosterPosition='Quarterback', PlayerDay='1', PlayerGame='1', StadiumType='Outdoor', FieldType='Synthetic', Temperature='63', Weather='Clear and warm', PlayType='Pass', PlayerGamePlay='2', Position='QB', PositionGroup='QB'),
 Row(PlayerKey='26624', GameID='26624-1', PlayKey='26624-1-3', RosterPosition='Quarterback', PlayerDay='1', PlayerGame='1', StadiumType='Outdoor', FieldType='Synthetic', Temperature='63', Weather='Clear and warm', PlayType='Rush', PlayerGamePlay='3', Position='QB', PositionGroup='QB'),
 Row(PlayerKey='26624', GameID='26624-1', PlayKey='26624-1-4', RosterPosition='Quarterback', PlayerDay='1', PlayerGame='1', StadiumType='Ou

In [16]:
# remove header, split str to list
# injury_rec = injury_rec.filter(lambda x: 'PlayerKey' not in x).map(lambda x: x.split(','))
# play_rec = play_rec.filter(lambda x: 'PlayerKey' not in x).map(lambda x: x.split(','))

In [23]:
# rdd->df
# injury_rec_df = injury_rec.toDF(['PlayerKey','GameID','PlayKey','BodyPart','Surface','DM_M1','DM_M7','DM_M28','DM_M42'])
# play_rec_df = play_rec.toDF(['_PlayberKey','_GameID','_PlayKey','RosterPosition','PlayerDay','PlayerGame','StadiumType','FieldType','Temperature','Weather','PlayType','PlayerGamePlay','Position','PositionGroup']

In [72]:
# join table
injury_play_rec_df = injury_rec_df.join(play_rec_df, injury_rec_df.PlayKey == play_rec_df.PlayKey,'left')

In [73]:
injury_play_rec_df.show()

+---------+--------+-----------+--------+---------+-----+-----+------+------+---------+--------+-----------+-----------------+---------+----------+-----------------+---------+-----------+--------------+--------+--------------+--------+-------------+
|PlayerKey|  GameID|    PlayKey|BodyPart|  Surface|DM_M1|DM_M7|DM_M28|DM_M42|PlayerKey|  GameID|    PlayKey|   RosterPosition|PlayerDay|PlayerGame|      StadiumType|FieldType|Temperature|       Weather|PlayType|PlayerGamePlay|Position|PositionGroup|
+---------+--------+-----------+--------+---------+-----+-----+------+------+---------+--------+-----------+-----------------+---------+----------+-----------------+---------+-----------+--------------+--------+--------------+--------+-------------+
|    44860| 44860-5| 44860-5-52|    Knee|  Natural|    1|    1|     0|     0|    44860| 44860-5| 44860-5-52|       Linebacker|       29|         5|          Outdoor|  Natural|         67|        Cloudy|    Rush|            52|     OLB|           LB|


In [77]:
cnt = injury_play_rec_df.count()
print('No. of injury record: ',cnt)

No. of injury record:  105


In [84]:
# the position of the injuried player
injury_pos_result = injury_play_rec_df.groupBy(injury_play_rec_df.Position).count()
injury_pos_result = injury_pos_result.collect()
print(injury_pos_result)

[Row(Position='OLB', count=12), Row(Position=None, count=28), Row(Position='T', count=2), Row(Position='ILB', count=3), Row(Position='DB', count=1), Row(Position='FS', count=5), Row(Position='LB', count=2), Row(Position='MLB', count=4), Row(Position='RB', count=6), Row(Position='DE', count=5), Row(Position='C', count=4), Row(Position='TE', count=2), Row(Position='DT', count=2), Row(Position='WR', count=16), Row(Position='CB', count=8), Row(Position='SS', count=5)]


In [82]:
# the position group of the injuried player
injury_pos_gp_result = injury_play_rec_df.groupBy(injury_play_rec_df.PositionGroup).count()
injury_pos_gp_result = injury_pos_gp_result.collect()
print(injury_pos_gp_result)

[Row(PositionGroup='OL', count=6), Row(PositionGroup=None, count=28), Row(PositionGroup='DB', count=19), Row(PositionGroup='LB', count=21), Row(PositionGroup='DL', count=7), Row(PositionGroup='RB', count=6), Row(PositionGroup='TE', count=2), Row(PositionGroup='WR', count=16)]


In [85]:
# the position group of the injuried player
injury_roster_pos_result = injury_play_rec_df.groupBy(injury_play_rec_df.RosterPosition).count()
injury_roster_pos_result = injury_roster_pos_result.collect()
print(injury_roster_pos_result)

[Row(RosterPosition='Linebacker', count=21), Row(RosterPosition=None, count=28), Row(RosterPosition='Defensive Lineman', count=7), Row(RosterPosition='Offensive Lineman', count=6), Row(RosterPosition='Running Back', count=6), Row(RosterPosition='Safety', count=11), Row(RosterPosition='Wide Receiver', count=16), Row(RosterPosition='Cornerback', count=8), Row(RosterPosition='Tight End', count=2)]


In [93]:
def writeResult(res, name):
    with open('../out/{}.txt'.format(name),'w') as f:
        for line in res:
            f.write('{} {} {}\n'.format(line[0],line[1],line[2]))

In [91]:
writeResult(injury_pos_result,'injury_pos_result')
writeResult(injury_pos_gp_result,'injury_pos_gp_result')
writeResult(injury_roster_pos_result,'injury_roster_pos_result')

In [92]:
# the position and body part of the injuried player
injury_pos_body_result = injury_play_rec_df.groupBy([injury_play_rec_df.Position,injury_play_rec_df.BodyPart]).count()
injury_pos_body_result = injury_pos_body_result.collect()
print(injury_pos_body_result)

[Row(Position='TE', BodyPart='Foot', count=1), Row(Position='T', BodyPart='Foot', count=1), Row(Position='DT', BodyPart='Knee', count=2), Row(Position='SS', BodyPart='Knee', count=3), Row(Position='WR', BodyPart='Knee', count=7), Row(Position='RB', BodyPart='Knee', count=4), Row(Position='MLB', BodyPart='Knee', count=1), Row(Position='CB', BodyPart='Knee', count=2), Row(Position='DE', BodyPart='Knee', count=2), Row(Position='CB', BodyPart='Ankle', count=5), Row(Position='DB', BodyPart='Knee', count=1), Row(Position='DE', BodyPart='Ankle', count=2), Row(Position='MLB', BodyPart='Ankle', count=3), Row(Position='WR', BodyPart='Ankle', count=8), Row(Position='FS', BodyPart='Ankle', count=3), Row(Position='CB', BodyPart='Foot', count=1), Row(Position=None, BodyPart='Foot', count=1), Row(Position='TE', BodyPart='Knee', count=1), Row(Position='C', BodyPart='Ankle', count=3), Row(Position=None, BodyPart='Knee', count=12), Row(Position='ILB', BodyPart='Ankle', count=1), Row(Position='RB', BodyP

In [94]:
writeResult(injury_pos_body_result,'injury_pos_body_result')